In [1]:
from enum import Enum 

import re 

In [2]:
direct_copy_file = '/home/kuugu/Projects/manner/backend/data.txt'

metadata_dict = {}

In [3]:
data_str = '' 

with open(direct_copy_file, 'r') as f:
    data_str = f.read() 


In [5]:
# split the pages 
page_strs = re.split('----P[0-9][0-9][0-9]', data_str)
page_strs = list(map(lambda x: x.strip(), page_strs))

progress_percent = 100 * sum(map(lambda x: 0 if len(x) == 0 else 1, page_strs))/len(page_strs)
progress_percent = round(progress_percent, 2)

metadata_dict['project_progress_pct'] = progress_percent

print (metadata_dict['project_progress_pct'])

0.72


In [5]:
# parsing rules: 
# [R1] first line should be entirely in kannada, they are all kannada words 
# [R2] first line contains the word, if multiple forms are present they are comma separated 
# [R3] if the word is a verb, it might have the root word. root word is always in (brackets)
# [R4] second line can be the type of word (verb, noun, etc..), if it's not, it is the start of the meaning
# [R5] there cannot be only one line, this means there's no meaning provided 

word_type = {
    's.': 'substantive', 
    'v.a.': 'verb active', 
    'v.n.': 'verb neuter', 
    'vlg.': 'vulgar word', 
    'fg.': 'figurative', 
}

class ParseOptions(Enum): 
    EXPAND = 1 
    NO_EXPAND = 2 

class ParseOutputType(Enum): 
    EMPTY = 1 
    NO_DATA = 2 
    NOT_KANNADA = 3  
    PARSE_SUCCESS = 4 

def is_kannada(word): 
    for ch in word: 
        if not ((0x0C80 <= ch <= 0x0CFF) or (ch in (' ', ')', '('))): 
            return False
    return True 

def flatten_list(nested_list: list): 
    ret_list = [] 

    if len(nested_list) == 0:
        return ret_list 
    else: 
        for elem in nested_list: 
            if type(elem) == list:
                ret_list = ret_list + flatten_list(elem) 
            else: 
                ret_list.append(elem) 
    return ret_list


def get_kannada_words(line) -> list[str]: 
    kannada_words = [] 

    parsed_line = line.split()
    parsed_line = flatten_list(list(map(lambda x: x.split(','), parsed_line)))
    parsed_line = list(filter(lambda x: len(x)>0, parsed_line))

    # NOTE (kuugu): I am assuming the data is correct here 
    #               we need to build the CFG checker later 

    if len(parsed_line) == 1: 
        kannada_words = [parsed_line[0]]
    
    if len(parsed_line) > 1: 
        if parsed_line[-1][0] == '(': 
            kannada_words = parsed_line[:-1]

    return kannada_words

def parse_word_no_expand(word_data:str) -> list: 
    lines = word_data.strip().split('\n') 
    lines = list(map(lambda x: x.strip(), lines))
        
    if len(lines) == 1 and lines[0] == '':
        return [{'parse_output_type': ParseOutputType.EMPTY}]
    elif len(lines) == 1: 
        if not is_kannada(lines[0]):
            return [{'word': lines[0], 'parse_output_type': ParseOutputType.NOT_KANNADA}] # R1 
        else: 
            return [{'word': lines[0], 'parse_output_type': ParseOutputType.NO_DATA}] # R5 

    kannada_words = get_kannada_words(lines[0])

    ret_data = [] 

    for word in kannada_words:
        ret_data.append(
            {
                'word': word, 
                'parse_output_type': ParseOutputType.PARSE_SUCCESS, 
                'data': '\n'.join(lines[1:])
            }
        )
    
    return ret_data 

def parse_word(word_data, parse_type=ParseOptions.NO_EXPAND):
    if parse_type == ParseOptions.EXPAND:
        raise NotImplementedError

    if parse_type == ParseOptions.NO_EXPAND: 
        return parse_word_no_expand(word_data)

    if word_data.strip() == '': 
        return [{'parse_output_type': ParseOutputType.EMPTY}]

 

In [8]:
def test_flatten_list():
    test_list = [
        1, 
        [
            [
                [
                    1, 2, [
                        3, 4]
                ], "hello" 
            ]
        ], "hi", 3, 4, []
    ]

    assert (flatten_list(test_list) == [1, 1, 2, 3, 4, 'hello', 'hi', 3, 4]) 

def test_get_kannada_words(): 
    x = get_kannada_words('ಅಂಗೋಪಾಂಗ,ಪಾಂಗ (ಅಂಗ-ಉಪ-ಅಂಗ)')
    assert(len(x) == 2) 
    assert(x == ['ಅಂಗೋಪಾಂಗ', 'ಪಾಂಗ'])

test_flatten_list()
test_get_kannada_words() 

In [17]:
word_cnt = 0

for page_str in page_strs: 
    words_data = page_str.split('\n\n')

    # print (words_data)

    # for word_data in words_data[:3]: 
    #     print (parse_word(word_data)) 

    for word_data in words_data: 
        for word in parse_word(word_data): 
            if word['parse_output_type'] == ParseOutputType.PARSE_SUCCESS: 
                word_cnt += 1 
print (word_cnt) 

63
